# Building a simple RAG chatbot with LangChain, Hugging Face, FAISS, Amazon SageMaker and Amazon Textract

In [ ]:
%%sh
pip install sagemaker langchain amazon-textract-caller amazon-textract-textractor sentence-transformers pypdf 
pip install faiss-cpu -U

In [1]:
import boto3, json, sagemaker
from typing import Dict
from langchain import LLMChain
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.llms import SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/studio-lab-user/.config/sagemaker/config.yaml


## Deploy LLM on SageMaker

In [2]:
from sagemaker import get_execution_role
sagemaker_session = sagemaker.Session()
role = 'arn:aws:iam::182943155813:role/service-role/AmazonSageMaker-ExecutionRole-20240214T125955'

hub = {
	'HF_MODEL_ID':'mistralai/Mistral-7B-Instruct-v0.1',
	'SM_NUM_GPUS': '1'
}

huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="1.1.0"),
	env=hub,
	role=role 
)

predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g5.2xlarge",
	container_startup_health_check_timeout=300,
  )

---------!

In [3]:
endpoint_name = predictor.endpoint_name
endpoint_name

'huggingface-pytorch-tgi-inference-2024-03-07-08-57-28-692'

## Configure LLM in LangChain

In [4]:
model_kwargs = {"max_new_tokens": 512, "top_p": 0.8, "temperature": 0.8}

In [5]:
class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: Dict) -> bytes:
        input_str = json.dumps(
            # Mistral prompt, see https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1
            {"inputs": f"<s>[INST] {prompt} [/INST]", "parameters": {**model_kwargs}}
        )
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        splits = response_json[0]["generated_text"].split("[/INST] ")
        return splits[1]

content_handler = ContentHandler()

In [6]:
sm_client = boto3.client("sagemaker-runtime") # needed for AWS credentials

llm = SagemakerEndpoint(
    endpoint_name=endpoint_name,
    model_kwargs=model_kwargs,
    content_handler=content_handler,
    client=sm_client,
)

## Zero-shot example

In [7]:
system_prompt = """
As a materials science assistant, please answer the question, focusing on numerical data.
Don't invent facts. If you can't provide a factual answer, say you don't know what the answer is.
"""

prompt = PromptTemplate.from_template(system_prompt + "{content}")

In [8]:
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [9]:
question = "Provide me a step-by-step process to produce high-quality monolayer graphene. Provide the source."

query = f"question: {question}"

In [13]:
answer = llm_chain.invoke({query})
print(answer)

{'content': {'question: Provide me a step-by-step process to produce high-quality monolayer graphene. Provide the source.'}, 'text': 'To produce high-quality monolayer graphene, the following steps can be followed:\n\n1. Exfoliation: The first step is to exfoliate graphite to obtain a single layer of graphene. This can be done using a variety of methods, such as mechanical exfoliation, chemical exfoliation, or thermal exfoliation.\n2. Purification: Once the graphene has been exfoliated, it needs to be purified to remove any impurities or defects that may be present. This can be done using a variety of methods, such as chemical purification or physical purification.\n3. Characterization: The purified graphene needs to be characterized to ensure that it is of high quality. This can be done using a variety of techniques, such as atomic force microscopy (AFM), Raman spectroscopy, or scanning tunneling microscopy (STM).\n4. Transfer: Once the graphene has been characterized, it needs to be 

## RAG example with PDF files

In [14]:
from langchain.document_loaders import AmazonTextractPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA

In [15]:
# Define S3 bucket and prefix for PDF storage

bucket = sagemaker.Session().default_bucket()
prefix = "graphene-rag"

In [16]:
%%sh -s $bucket $prefix
aws s3 cp --recursive pdfs s3://$1/$2/

upload: pdfs/graphenepdfs_full.pdf to s3://sagemaker-us-east-1-182943155813/graphene-rag/graphenepdfs_full.pdf


In [17]:
# Build list of S3 URIs

s3 = boto3.client("s3")
objs = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
objs = objs['Contents']
uris = [f's3://{bucket}/{obj["Key"]}' for obj in objs]
uris    

['s3://sagemaker-us-east-1-182943155813/graphene-rag/graphenepdfs_full.pdf']

### Analyze documents with Amazon Textract and split them in chunks

In [18]:
%%time

textract_client = boto3.client('textract')
splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=0)

all_chunks = []

for uri in uris:
    loader = AmazonTextractPDFLoader(uri, client=textract_client)
    document = loader.load()
    chunks = splitter.split_documents(document)
    all_chunks += chunks
    print(f"Loaded {uri}, {len(document)} pages, {len(chunks)} chunks")

Loaded s3://sagemaker-us-east-1-182943155813/graphene-rag/graphenepdfs_full.pdf, 100 pages, 1396 chunks
CPU times: user 8.51 s, sys: 240 ms, total: 8.75 s
Wall time: 1min 3s


### Embed document chunks and store them in FAISS
https://github.com/facebookresearch/faiss 

In [19]:
# Define embedding model
# See https://huggingface.co/spaces/mteb/leaderboard

embedding_model_id = "BAAI/bge-small-en-v1.5"

embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model_id,
)

In [20]:
%%time
# Embed chunks
embeddings_db = FAISS.from_documents(all_chunks, embeddings)

CPU times: user 48.4 s, sys: 810 ms, total: 49.2 s
Wall time: 27.8 s


In [21]:
# Save database
embeddings_db.save_local("faiss_index")

### Shortcut : load existing embedding database

In [22]:
embeddings_db = FAISS.load_local("faiss_index", embeddings)

********

### Configure RAG chain

In [23]:
retriever = embeddings_db.as_retriever(search_kwargs={"k": 10})

In [24]:
# Define prompt template
prompt_template = """
As a materials science assistant, please answer the question, focusing on numerical data. Answer naturally like speaking to a human. 
Do NOT invent any facts. If you can't provide a factual answer, say you don't know what the answer is. 
Even if source is not asked, always provide the author, title of paper and year of publication belonging to the embedding used in the response generation, placing them in a continuous sentence in the following format: 'Source: Author, Title of paper (Year of Publication)'.
All sources should be from FAISS database.

question: {question}

important context to prioritise your built-in knowledge: {context}

"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [25]:
chain = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff",
    retriever=retriever, 
    chain_type_kwargs = {"prompt": prompt})

### Ask our question again

In [26]:
%%time
question = "Provide me a step-by-step process to produce high-quality monolayer graphene. Provide the source."
answer = chain.run({"query": question})
print(answer)

To produce high-quality monolayer graphene using thermal CVD, the following steps can be taken:

1. Start by heating a copper (Cu) foil (10 x 10 cm2) to 1000 °C in a quartz tube for 60 minutes.
2. Once the Cu foil is heated, remove it from the quartz tube and let it cool down to room temperature.
3. Place the Cu foil in a chamber and expose it to ethylene gas at a base pressure of 1x10-10 mbar.
4. Heat the chamber to a temperature of 400-500 °C, and leave it at this temperature for several hours.
5. After the exposure to ethylene gas, remove the Cu foil from the chamber and let it cool down to room temperature.
6. Use scanning electron microscopy (SEM) or atomic force microscopy (AFM) to examine the surface of the Cu foil and verify that it has grown a single layer of graphene.

Source: Bointon, T. H., High Quality Monolayer Graphene Synthesized by Resistive Heating Cold Wall Chemical Vapor Deposition, Journal of Applied Materials and Interfaces, 2015.
CPU times: user 161 ms, sys: 248 

In [27]:
%%time
question = "Give me the summary of preparing graphene using the dry transfer method."
answer = chain.run({"query": question})
print(answer)

The dry transfer method for preparing graphene involves depositing the graphene on a substrate using a roll-to-roll process. The graphene is then transferred onto the substrate using the standard wet transfer method. The flow of the fabrication process for the double-layer graphene stack is shown in Supplementary Fig. 10. The method of producing graphene involves depositing 72 nm of graphene on a substrate using a thermal evaporator and then transferring it onto the substrate using a semi-dry transfer method. The yield of graphene produced using this method is estimated by weighting the dried graphene powders and the dried sediment. The yield of graphene produced using this method is about 15%. The paper by Liu et al. (2018) presents a systematic study to explore the fundamental factors that affect the yield of graphene produced using this method. The paper by Juang et al. (2010) presents a fabrication scheme for clean suspended structures using chemical-vapor-deposition-grown graphene

In [28]:
%%time
question = "Describe the materials used for graphene produced using CVD."
answer = chain.run({"query": question})
print(answer)

The materials used for graphene produced using CVD typically consist of a substrate material, such as copper (Cu), and a gas mixture, including hydrocarbons and catalytic agents. The substrate material is used to support the growth of the graphene film, while the gas mixture provides the necessary atoms and molecules for the formation of graphene.

The quality and properties of the graphene produced by CVD can be influenced by various parameters, such as temperature, pressure, and gas composition. For example, increasing the temperature of the substrate material can improve the catalytic activity, leading to the synthesis of high-quality graphene.

Recent studies have focused on improving the mass production of CVD graphene films, including the development of more efficient and cost-effective methods. For example, a recent study demonstrated the growth of high-quality graphene using resistive-heating cold-wall CVD, which is 100 times faster and 99% lower cost than standard CVD (Deng, 2

In [29]:
%%time
question = "Should I expect monolayer or few-layered graphene when produced using chemical reduction of exfoliated graphite oxide?"
answer = chain.run({"query": question})
print(answer)

Based on the provided information, it is likely that the graphene material produced using chemical reduction of exfoliated graphite oxide will consist of few-layered graphene, with a number of layers ranging from 4 to 10. The presence of multi-layer graphene was also confirmed in some cases, but the number of layers was greater than 3. The yield of few-layer graphene was found to be around 11.3 wt.% when using wheat straw as starting material. However, it's worth noting that the number of layers may vary depending on the specific parameters used in the production process.
CPU times: user 114 ms, sys: 3.78 ms, total: 117 ms
Wall time: 5.05 s


In [30]:
%%time
question = "What is a suggested number of graphite flakes to synthesize graphene?"
answer = chain.run({"query": question})
print(answer)

The suggested number of graphite flakes to synthesize graphene varies depending on the method used. However, for the annealing method, a typical range is between 2-5 g of graphite flakes. This method involves oxidizing the graphite flakes and then intercalating them in a solvent solution to produce graphene. The amount of graphite flakes needed depends on the size of the flakes and the desired yield of graphene. For example, in one study, 2 g of graphite flakes were used to produce 0.5 g of graphene[114,147,148,192].

Source: Liu, Z., et al. "A facile and scalable method for the production of high-quality graphene by annealing of graphite flakes." Carbon, vol. 147, no. 148, 2019, pp. 114-148.
CPU times: user 91.8 ms, sys: 64 µs, total: 91.8 ms
Wall time: 8.14 s


In [31]:
%%time
question = "Give me the differences in using laser or CVD to synthesize graphene. Present the information in a table with fixed headers ‘Laser’ and ‘CVD’."
answer = chain.run({"query": question})
print(answer)

Here is the table you requested:

| Parameter | Laser | CVD |
| --- | --- | --- |
| Growth rate | Fast | Slow |
| Cost | High | Low |
| Quality of graphene | Poor | Good |
| Layers of graphene | Single, few, multi | Single, few, multi |

Note: The quality of graphene synthesized using CVD is generally better than that synthesized using laser, and the cost of CVD is significantly lower. The growth rate of CVD is slower, but it can be controlled by varying the substrate temperature and other parameters. CVD can be used to synthesize single, few, and multi-layer graphene, while laser can only produce single-layer graphene.

Source:

* Campo, T. (2016). Graphene Synthesis by Plasma-Enhanced CVD Growth with Ethanol. AJEASSP, 574, 583. doi: 10.3844/ajeassp.2016.574.583
* Ghaemi, F., & Ghaemi, A. (2016). Synthesis of Different Layers of Graphene on Stainless Steel Using the CVD Method. Journal of Nanoscience and Nanotechnology, 11(1), 1-8. doi: 10.1186/s11671-016-1709-x
* Yamada, T. (2013). L

In [32]:
%%time
question = "I think I remember reading an article on producing graphene with rice husks. Can you help me find the paper and if the produced graphene is a mono-layer or a multi-layer?"
answer = chain.run({"query": question})
print(answer)

Based on the information provided, the paper that you are thinking of is "Synthesis and characterization of graphene derived from rice husks" by Mohammad Shafri Ismail, published in 2018. According to the paper, the produced graphene is multi-layered (more than 5 layers).
CPU times: user 133 ms, sys: 5.92 ms, total: 139 ms
Wall time: 2.82 s


In [33]:
%%time
question = "I need the steps to produce graphene for solar cells."
answer = chain.run({"query": question})
print(answer)

To produce graphene for solar cells, the following steps can be followed:

1. Preparation of the substrate: A copper foil (10 X 10 cm2) can be used as the substrate.
2. Heating: The copper foil can be heated to 1000 °C for 60 min in a quartz tube using thermal CVD.
3. Cooling: The graphene film can be cooled down after the heating process.

Source: Author, Title of paper (Year of Publication)

Method of producing graphene: Monolayer graphene was synthesized using thermal CVD. First, a copper (Cu) foil (10 X 10 cm2) was placed in a quartz tube and heated to 1000 °C for 60 min with methane as a carbon source. The synthesis process was performed at a relatively low synthesis temperature.

Method of producing graphene: The synthesis process of rice husk-derived graphene was performed using a commercially available cold-wall chemical vapor deposition (CVD) gas and extensive vacuum processing to create a highly controlled environment, isolated from ambient air, to enable the growth of graphe

In [34]:
%%time
question = "Tell me a method to synthesise graphene involving a compressing machine. "
answer = chain.run({"query": question})
print(answer)

In order to synthesize graphene using a compressing machine, it is important to note that this method is not a commonly used method for producing graphene. However, there have been some studies that have investigated the use of a compressing machine for graphene synthesis.

One such study is titled "A facile scalable method to produce highly porous graphene from reduced graphene oxide via thermal annealing" by Xue et al. (2015). In this study, the authors used a thermal annealing process to produce highly porous graphene from reduced graphene oxide. The process involved heating the reduced graphene oxide to a temperature of 500°C for 2 hours, which resulted in the formation of highly porous graphene with a surface area of 1000 m2/g.

Another study that investigated the use of a compressing machine for graphene synthesis is titled "Efficient large-scale synthesis of few-layered graphene using a buffer gas-assisted arc discharge method" by Chen et al. (2014). In this study, the authors u

In [35]:
%%time
question = "Give me a process to synthesize graphene that doesn’t use exfoliation. Provide the source."
answer = chain.run({"query": question})
print(answer)

To synthesize graphene without exfoliation, one method is to use the controlled electrochemical exfoliation of graphite in an aqueous ammonium sulfate electrolyte. This method was reported by Guoxiu Wang in the paper "Highly efficient and large-scale synthesis of graphene by electrolytic exfoliation" published in the journal Carbon in 2009. The method involves the use of high purity graphite rods, which are exfoliated to produce water-dispersible graphene with high quality and large quantity. This method provides an industrially scalable route for new potential applications of graphene-based materials.
CPU times: user 116 ms, sys: 4.02 ms, total: 120 ms
Wall time: 5.57 s


In [36]:
%%time
question = "What is the most recent graphene synthesis experiment available?"
answer = chain.run({"query": question})
print(answer)

The most recent graphene synthesis experiment available is the one by ZiZhao Xu, titled "Bulk-scale synthesis of randomly stacked graphene with high crystallinity" published in the journal Carbon in 2021. The study focuses on the production of high-quality graphene on a large scale, at an affordable cost, and in a reproducible manner. The synthesis process involves chemical vapor deposition and transfer by a roll-to-roll process. The study shows that graphene can be synthesized at a relatively low synthesis temperature, which is a significant advancement in the field. The authors also report the presence of single-, few-, and multi-layer graphene in their synthesis process.
CPU times: user 69.5 ms, sys: 0 ns, total: 69.5 ms
Wall time: 6.04 s


In [37]:
%%time
question = "Give me three ways to produce graphene from graphite. Provide the source for each way."
answer = chain.run({"query": question})
print(answer)

There are several ways to produce graphene from graphite. Here are three methods, along with their respective sources:

1. Exfoliation method: This method involves the use of mechanical or chemical processes to separate graphite layers, resulting in the formation of graphene. A simple and easy process for producing graphene using this method was reported in a study by Marcano et al. (2010). The study presented a modified Hummers' method for preparing graphene oxide samples from natural graphite powder.
2. Chemical vapor deposition (CVD) method: This method involves the decomposition of a gas mixture in the presence of a substrate material, resulting in the formation of a thin film of graphene. The CVD synthesis of graphene was carried out in a 100 mm quartz tube of 1400 mm in length, as described in a study by Al-Johani et al. (2012). The study used a three-zones furnace of 760 mm length to surround the reactor.
3. Hummers' method: This method involves the oxidation of graphite using a

In [38]:
%%time
question = "I need to produce graphene inks. Give me the step-by-step method."
answer = chain.run({"query": question})
print(answer)

To produce graphene inks, the following steps can be taken:

1. Monolayer graphene can be synthesized using thermal chemical vapor deposition (CVD) (Source: Huang, et al. (2011)). First, a copper (Cu) foil (10 x 10 cm^2) can be placed in a quartz tube and heated to 1000 °C for 60 min.
2. The synthesized graphene can then be dispersed in a solvent to form a graphene ink. This can be done using a variety of techniques, such as mechanical exfoliation or sonication (Source: Zhang, et al. (2011)).
3. The graphene ink can be used to print conductive stripes on flexible substrates, such as polyethylene (Source: Liu, et al. (2010)).

It's worth noting that the production of high-quality graphene on a large scale, at an affordable cost, and in a reproducible manner is still a challenge in the field of graphene fabrication (Source: Bao, et al. (2011)).
CPU times: user 132 ms, sys: 3.9 ms, total: 136 ms
Wall time: 9.82 s


In [39]:
%%time
question = "Is there a paper describing how to exfoliate graphene with a high shear mixer?"
answer = chain.run({"query": question})
print(answer)

Yes, there is a paper describing how to exfoliate graphene with a high shear mixer. The paper "Reliable Exfoliation of Large-Area High-Quality Flakes of Graphene and Other Two-Dimensional Materials" by Yuan Huang, published in 2015, describes a method for producing large-area, high-quality flakes of graphene and other two-dimensional materials using a high shear mixer. The authors used a Silverson L5M rotor-stator mixer to exfoliate graphite and achieved high conductivity graphene patterns on various substrates. The paper suggests that the modified exfoliation method is effective for producing high-quality flakes of a wide range of 2D materials.
CPU times: user 118 ms, sys: 1.75 ms, total: 120 ms
Wall time: 6.3 s


In [40]:
%%time
question = "I am using glucose and ammonium salts to synthesize graphene. How will the graphene look like, and how many layers of graphene is the result?"
answer = chain.run({"query": question})
print(answer)

Based on the information provided, the graphene synthesized using glucose and ammonium salts will have multiple layers, typically greater than 3 layers. However, the specific number of layers will depend on the parameters used in the synthesis process, such as the temperature, time, and pressure.

The CVD method is commonly used to synthesize graphene, and the number of layers produced can vary depending on the specific conditions used. In some studies, single-layer graphene has been produced, while in others, few- and multi-layer graphene has been synthesized.

The number of layers in graphene can be confirmed using various techniques such as diffraction, atomic force microscopy, and Raman spectroscopy.

In the case of the Mettler Toledo (AX105 Delta Range) balance, the accuracy of +0.0003 g is not relevant to the number of layers in the graphene produced.

Overall, the number of layers in graphene produced using glucose and ammonium salts will depend on the specific synthesis process

In [41]:
%%time
question = "I have a MW PECVD set-up, how do I produce FLG?"
answer = chain.run({"query": question})
print(answer)

To produce FLG using a MW PECVD set-up, you can use a method described in a study by Kang et al. (2015). The study demonstrates the transfer of centimeter-scale FLG from Ni foil to transparent flexible polyethylene terephthalate substrates via an efficient roll-to-roll process. The Ni foil (30 µm in thickness) was used as the substrate and the detailed description of the experimental procedure for GO synthesis is given in the Supplementary data. The current experimental setup can produce ca. 17 g of GO using 12 g of natural flake graphite with full coverage by atmospheric pressure CVD implemented in a vacuum-free outfit. The absence of FLG patches is related to the absence of decomposition of methane on the backside microwave plasma chemical vapor deposition (MPCVD). The plasma/metal interaction can be advantageous for a rapid synthesis of such thin films. The process can produce films of the MPCVD deposition of thin carbon films on Cu foil, and the findings open new pathways for a rap

In [42]:
%%time
question = "How do I dry graphene and under what parameters? Provide a paper describing the drying process."
answer = chain.run({"query": question})
print(answer)

The paper "Dry transfer method for suspended graphene on lift-off-resist: simple ballistic devices with Fabry-P'erot interference" by Ying Liu (2018) describes a method for producing graphene by thermal evaporation of copper films on a quartz substrate. The resulting graphene was then transferred to a lift-off-resist using a simple ballistic device with Fabry-P'erot interference. The dried graphene was obtained by a dry transfer method using a sacrificial polymeric support.

Source: Liu, Y., (2018). Dry transfer method for suspended graphene on lift-off-resist: simple ballistic devices with Fabry-P'erot interference. arXiv:1903.03780 [Physics, condensed matter]. doi:10.48550/arXiv.1903.03780
CPU times: user 117 ms, sys: 5 µs, total: 117 ms
Wall time: 7.98 s


In [43]:
%%time
question = "Provide a list of the parameters involved in a graphene synthesis involving the Hummer’s method."
answer = chain.run({"query": question})
print(answer)

The parameters involved in a graphene synthesis using the Hummer's method are:

1. Graphite powder (<44 mm, 99.99%, supplied by Sigma-Aldrich)
2. Sodium nitrate (0.5 g)
3. Concentrated H2SO4 (70 mL)
4. Modified Hummer's method
5. Temperature (typically around 90-100°C)
6. Time (typically around 2-4 hours)
7. pH (typically around 0.5)
8. Oxidant (typically around 10-15 mL)
9. Concentration of oxidant (typically around 10-15 mL)
10. Mechanical exfoliation (typically around 1.5 nm thickness)

Source: Marcano et al., 2010. Synthesis of Graphene Oxide from Graphite Powder. Materials Research Express, 1(1), 1-6.
CPU times: user 132 ms, sys: 3.76 ms, total: 136 ms
Wall time: 9.11 s


In [46]:
%%time
question1 = "Describe the facile liquid phase exfoliation method with tested and tried amounts of materials. Provide the source."
answer1 = chain.run({"query": question1})
print(answer1)

question2 = "Will the suggested material amounts produce a monolayer or multiple layers of graphene?"
answer2 = chain.run({"query": question2})
print(answer2)

The facile liquid phase exfoliation method has been tested and tried with amounts of materials, specifically with graphite flakes. The method involves pre-treating the graphite foil with liquid nitrogen and absolute ethanol, and then mixing it with a 9:1 mixture of concentrated H2SO4/H3PO4 and 18 g of KMnO4. The mixture is stirred mechanically at 35-40 C for 4e5 h. The resulting exfoliated graphene is labeled as EG and can be further dispersed in organic solvents or aqueous solution with a conventional ammonium sulfate electrolyte. The method has been tested using a home-built setup with rolled graphite foils as working electrodes and platinum foils as counter electrodes. The source for this information is: Source: Li, Q., Zhang, X., Zhang, Y., Zhang, Y., Wang, Y., Wang, Y., ... & Zhang, Y. (2015). Facile liquid phase exfoliation of defect-free graphene. Materials Research Express, 15(11), 13888-13895.
Based on the provided data, the suggested material amounts will produce a multi-laye

## Delete endpoint and model

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()